In [133]:
import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'



#!pip install thefuzz
#!pip install rapidfuzz
#!pip install python-Levenshtein

from rapidfuzz import process, fuzz

df_games = pd.read_csv("originales/games.csv")
df_teams = pd.read_csv("originales/teams.csv")
df_teamstats = pd.read_csv("originales/teamstats.csv")
df_leagues = pd.read_csv("originales/leagues.csv")
df_appearances = pd.read_csv("originales/appearances.csv")
df_players = pd.read_csv("originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("originales/shots.csv")



""" df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") """

' df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")\ndf_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")\ndf_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")\ndf_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")\ndf_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")\ndf_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")\ndf_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") '

In [134]:
df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
#df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    #df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
#)
#df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    #["homeProbability", "drawProbability", "awayProbability"], axis=1
#)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
)





In [135]:
df_apperance_merged_with_players.isna().sum()

gameID           0
playerID         0
goals            0
ownGoals         0
shots            0
xGoals           0
xGoalsChain      0
xGoalsBuildup    0
assists          0
keyPasses        0
xAssists         0
position         0
positionOrder    0
yellowCard       0
redCard          0
time             0
substituteIn     0
substituteOut    0
leagueID         0
name             0
dtype: int64

In [136]:
df2_appearances = pd.read_csv("players/appearances.csv")

df2_clubs = pd.read_csv("players/clubs.csv")
df2_competitions = pd.read_csv("players/competitions.csv")
df2_games = pd.read_csv("players/games.csv")
df2_player_val = pd.read_csv("players/player_valuations.csv")
df2_players = pd.read_csv("players/players.csv")


""" df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")
df2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")
df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") """


' df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")\ndf2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")\ndf2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")\ndf2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")\ndf2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")\ndf2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")\ndf2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") '

In [137]:
df2_appearances=df2_appearances.dropna()
df2_appearances.isna().sum()

appearance_id         0
game_id               0
player_id             0
player_club_id        0
date                  0
player_pretty_name    0
competition_id        0
yellow_cards          0
red_cards             0
goals                 0
assists               0
minutes_played        0
dtype: int64

In [138]:
top_5_ligas = df2_competitions.loc[
    (df2_competitions["pretty_name"] == "Bundesliga")
    | (df2_competitions["pretty_name"] == "Laliga")
    | (df2_competitions["pretty_name"] == "Premier League")
    | (df2_competitions["pretty_name"] == "Serie A")
    | (df2_competitions["pretty_name"] == "Ligue 1")
]


def match_ligue(x, col, score):
    match = process.extractOne(x, col, score_cutoff=score, scorer=fuzz.token_sort_ratio)
    if match:
        return match[2]
    else:
        return 2


# ---------------------------------------------------------------------------------------------------------
# top_5_ligas["pretty_name"]      df_teams.name
top_5_ligas["liga_new_id"] = df_leagues.loc[
    top_5_ligas["pretty_name"]
    .apply(lambda x: match_ligue(x, df_leagues.name, 50))
    .values,
    ["leagueID"],
].values
# ---------------------------------------------------------------------------------------------------------


df2_clubs_top5 = df2_clubs[
    df2_clubs["domestic_competition_id"].isin(top_5_ligas["competition_id"])
]


df2_games_top5 = df2_games[
    df2_games["competition_id"].isin(top_5_ligas["competition_id"])
]
seasons = df_f_game["season"].unique()


df2_games_top5 = df2_games_top5[df2_games_top5["season"].isin(seasons)]



In [139]:
df2_games_top5 = pd.merge(df2_games_top5, top_5_ligas, on="competition_id")


df2_appearances = df2_appearances[
    df2_appearances["game_id"].isin(df2_games_top5["game_id"])
]
#df2_players = df2_players[
    #df2_players["current_club_id"].isin(df2_clubs_top5["club_id"])
#]





#df2_players_val = df2_player_val[
    #df2_player_val["player_id"].isin(df2_players["player_id"])
#]


df2_games_top5["date"] = pd.to_datetime(df2_games_top5["date"], format="%Y/%m/%d")
df_f_game["date"] = pd.to_datetime(df_f_game["date"], format="%Y/%m/%d")


df2_games_top5["month_year"] = df2_games_top5["date"].dt.to_period("M")
df_f_game["month_year"] = df_f_game["date"].dt.to_period("M")

df_f_game["match_name"] = df_f_game["home_name"] + " vs " + df_f_game["away_name"]

nan_values = pd.DataFrame(np.nan, index=[0], columns=df_f_game.columns)

df_f_game = df_f_game.append(nan_values, ignore_index=True)
df_f_game['date'] = pd.to_datetime(df_f_game['date']).dt.date


C:\Users\Mati\AppData\Local\Temp\ipykernel_2024\3889618879.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_game = df_f_game.append(nan_values, ignore_index=True)


In [140]:
df2_appearances.isna().sum()

appearance_id         0
game_id               0
player_id             0
player_club_id        0
date                  0
player_pretty_name    0
competition_id        0
yellow_cards          0
red_cards             0
goals                 0
assists               0
minutes_played        0
dtype: int64

In [141]:
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Turin", "club_home_pretty_name"
] = "Torino"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Turin", "club_away_pretty_name"
] = "Torino"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Mailand"),
    "club_away_pretty_name",
] = "Ac Milan"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Mailand"),
    "club_home_pretty_name",
] = "Ac Milan"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Juventus"),
    "club_away_pretty_name",
] = "Juventus"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Juventus"),
    "club_home_pretty_name",
] = "Juventus"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Fc Granada"),
    "club_away_pretty_name",
] = "Granada"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Fc Granada"),
    "club_home_pretty_name",
] = "Granada"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Real Sociedad San Sebastian",
    "club_away_pretty_name",
] = "Real Sociedad"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Real Sociedad San Sebastian",
    "club_home_pretty_name",
] = "Real Sociedad"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Stoke City", "club_away_pretty_name"
] = "Stoke"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Stoke City", "club_home_pretty_name"
] = "Stoke"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Swansea City", "club_away_pretty_name"
] = "Swansea"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Swansea City", "club_home_pretty_name"
] = "Swansea"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ud Las Palmas", "club_home_pretty_name"
] = "Las Palmas"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ud Las Palmas", "club_away_pretty_name"
] = "Las Palmas"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ssc Neapel", "club_home_pretty_name"
] = "Napoli"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ssc Neapel", "club_away_pretty_name"
] = "Napoli"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hellas Verona", "club_home_pretty_name"
] = "Verona"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hellas Verona", "club_away_pretty_name"
] = "Verona"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Us Lecce", "club_home_pretty_name"
] = "Lecce"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Us Lecce", "club_away_pretty_name"
] = "Lecce"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Atalanta Bergamo",
    "club_home_pretty_name",
] = "Atalanta"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Atalanta Bergamo",
    "club_away_pretty_name",
] = "Atalanta"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Brescia Calcio", "club_home_pretty_name"
] = "Brescia"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Brescia Calcio", "club_away_pretty_name"
] = "Brescia"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ac Florenz", "club_home_pretty_name"
] = "Fiorentina"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ac Florenz", "club_away_pretty_name"
] = "Fiorentina"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Spal", "club_home_pretty_name"
] = "SPAL 2013"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Spal", "club_away_pretty_name"
] = "SPAL 2013"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "As Rom", "club_home_pretty_name"
] = "roma"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "As Rom", "club_away_pretty_name"
] = "roma"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Us Sassuolo", "club_home_pretty_name"
] = "Sassuolo"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Us Sassuolo", "club_away_pretty_name"
] = "Sassuolo"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Lazio Rom", "club_home_pretty_name"
] = "Lazio"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Lazio Rom", "club_away_pretty_name"
] = "Lazio"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Genua Cfc", "club_home_pretty_name"
] = "genoa"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Genua Cfc", "club_away_pretty_name"
] = "genoa"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Cagliari Calcio",
    "club_home_pretty_name",
] = "Cagliari"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Cagliari Calcio",
    "club_away_pretty_name",
] = "Cagliari"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sampdoria Genua",
    "club_home_pretty_name",
] = "Sampdoria"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sampdoria Genua",
    "club_away_pretty_name",
] = "Sampdoria"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Delfino Pescara 1936",
    "club_home_pretty_name",
] = "Pescara"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Delfino Pescara 1936",
    "club_away_pretty_name",
] = "Pescara"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "1 Fsv Mainz 05", "club_home_pretty_name"
] = "Mainz 05"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "1 Fsv Mainz 05", "club_away_pretty_name"
] = "Mainz 05"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Ingolstadt 04",
    "club_home_pretty_name",
] = "Ingolstadt"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Ingolstadt 04",
    "club_away_pretty_name",
] = "Ingolstadt"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sc Paderborn 07",
    "club_home_pretty_name",
] = "Paderborn"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sc Paderborn 07",
    "club_away_pretty_name",
] = "Paderborn"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sv Darmstadt 98",
    "club_home_pretty_name",
] = "Darmstadt"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sv Darmstadt 98",
    "club_away_pretty_name",
] = "Darmstadt"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Bayern Munchen",
    "club_home_pretty_name",
] = "Bayern Munich"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Bayern Munchen",
    "club_away_pretty_name",
] = "Bayern Munich"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "1 Fc Koln", "club_home_pretty_name"
] = "FC Cologne"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "1 Fc Koln", "club_away_pretty_name"
] = "FC Cologne"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Tsg 1899 Hoffenheim",
    "club_home_pretty_name",
] = "Hoffenheim"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Tsg 1899 Hoffenheim",
    "club_away_pretty_name",
] = "Hoffenheim"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Palermo Fc", "club_home_pretty_name"
] = "Palermo"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Palermo Fc", "club_away_pretty_name"
] = "Palermo"

#-----------------------------------------------------------------------------------------------

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Girondins Bordeaux", "club_home_pretty_name"
] = "Bordeaux"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Girondins Bordeaux", "club_away_pretty_name"
] = "Bordeaux"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Stade Rennes", "club_home_pretty_name"
] = "Rennes"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Stade Rennes", "club_away_pretty_name"
] = "Rennes"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Lorient", "club_home_pretty_name"
] = "Lorient"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Lorient", "club_away_pretty_name"
] = "Lorient"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Losc Lille", "club_home_pretty_name"
] = "Lille"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Losc Lille", "club_away_pretty_name"
] = "Lille"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ogc Nizza", "club_home_pretty_name"
] = "Nizza"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ogc Nizza", "club_away_pretty_name"
] = "Nizza"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Thonon Evian Grand Geneve Fc", "club_home_pretty_name"
] = "Evian Thonon Gaillard"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Thonon Evian Grand Geneve Fc", "club_away_pretty_name"
] = "Evian Thonon Gaillard"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Stade Reims", "club_home_pretty_name"
] = "Reims"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Stade Reims", "club_away_pretty_name"
] = "Reims"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ea Guingamp", "club_home_pretty_name"
] = "Guingamp"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ea Guingamp", "club_away_pretty_name"
] = "Guingamp"

df2_games_top5["match_name"] = (
    df2_games_top5["club_home_pretty_name"]
    + " vs "
    + df2_games_top5["club_away_pretty_name"]
)




In [142]:
df_f_game

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,...,fouls_away,corners_away,yellowCards_away,redCards_away,result_away,liga,home_name,away_name,month_year,match_name
0,81.0,1.0,2015.0,2015-08-08,89.0,82.0,1.0,0.0,0.2843,0.3999,...,12.0,2.0,3.0,0.0,L,Premier League,Manchester United,Tottenham,2015-08,Manchester United vs Tottenham
1,82.0,1.0,2015.0,2015-08-08,73.0,71.0,0.0,1.0,0.3574,0.3500,...,13.0,3.0,4.0,0.0,W,Premier League,Bournemouth,Aston Villa,2015-08,Bournemouth vs Aston Villa
2,83.0,1.0,2015.0,2015-08-08,72.0,90.0,2.0,2.0,0.2988,0.4337,...,13.0,2.0,2.0,0.0,D,Premier League,Everton,Watford,2015-08,Everton vs Watford
3,84.0,1.0,2015.0,2015-08-08,75.0,77.0,4.0,2.0,0.6422,0.2057,...,17.0,3.0,4.0,0.0,L,Premier League,Leicester,Sunderland,2015-08,Leicester vs Sunderland
4,85.0,1.0,2015.0,2015-08-08,79.0,78.0,1.0,3.0,0.1461,0.2159,...,20.0,4.0,0.0,0.0,W,Premier League,Norwich,Crystal Palace,2015-08,Norwich vs Crystal Palace
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,16132.0,5.0,2020.0,2021-05-23,177.0,176.0,1.0,2.0,0.3367,0.2999,...,6.0,2.0,1.0,0.0,W,Ligue 1,Reims,Bordeaux,2021-05,Reims vs Bordeaux
12677,16133.0,5.0,2020.0,2021-05-23,163.0,235.0,2.0,0.0,0.6719,0.2502,...,10.0,3.0,0.0,0.0,L,Ligue 1,Rennes,Nimes,2021-05,Rennes vs Nimes
12678,16134.0,5.0,2020.0,2021-05-23,175.0,181.0,0.0,1.0,0.3541,0.3010,...,10.0,3.0,0.0,0.0,W,Ligue 1,Saint-Etienne,Dijon,2021-05,Saint-Etienne vs Dijon
12679,16135.0,5.0,2020.0,2021-05-23,225.0,179.0,1.0,1.0,0.1748,0.4863,...,9.0,3.0,1.0,0.0,D,Ligue 1,Strasbourg,Lorient,2021-05,Strasbourg vs Lorient


In [143]:
df2_games_top5

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,country_name,country_latitude,country_longitude,domestic_league_code,name,confederation,url_y,liga_new_id,month_year,match_name
0,2475559,ES1,domestic_league,2014,38. Matchday,2015-05-23,1084,368,2,3,...,Spain,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Fc Malaga vs Fc Sevilla
1,2475556,ES1,domestic_league,2014,38. Matchday,2015-05-23,418,3709,7,3,...,Spain,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Real Madrid vs Fc Getafe
2,2475555,ES1,domestic_league,2014,38. Matchday,2015-05-23,1533,993,3,0,...,Spain,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Sd Eibar vs Fc Cordoba
3,2475557,ES1,domestic_league,2014,38. Matchday,2015-05-23,940,714,3,2,...,Spain,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Celta Vigo vs Espanyol Barcelona
4,2475558,ES1,domestic_league,2014,38. Matchday,2015-05-23,3302,1049,2,3,...,Spain,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Ud Almeria vs Fc Valencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,3433222,IT1,domestic_league,2020,4. Matchday,2020-10-18,416,1390,2,3,...,Italy,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Torino vs Cagliari
12677,3433216,IT1,domestic_league,2020,4. Matchday,2020-10-17,4083,506,1,1,...,Italy,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Fc Crotone vs Juventus
12678,3433190,IT1,domestic_league,2020,1. Matchday,2020-09-21,5,1025,2,0,...,Italy,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-09,Ac Milan vs Fc Bologna
12679,3433206,IT1,domestic_league,2020,3. Matchday,2020-10-04,4171,1025,1,0,...,Italy,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Benevento Calcio vs Fc Bologna


In [144]:
#----------------------------------------------------------
# despues de crear las columnas de los nombres de los match
# comparo los match de df_game con df2_games_top5
#---------------------------------------------------------

def match_games(x, y, score):

    liga = x["liga_new_id"]
    date = x["date"]
   

    y = y.loc[
        (y["leagueID"] == liga)
        & (y["date"] == date)
    
    ]

    match = process.extractOne(
        x["match_name"], y["match_name"], score_cutoff=score, scorer=fuzz.WRatio
    )

    if match:
        return match[2]
    else:
        return 12680


# ---------------------------------------------------------------------------------------------------------

df2_games_top5[["match_name_df", "homeTeamID", "awayTeamID",'home_name']] = df_f_game.loc[
    df2_games_top5.apply(lambda x: match_games(x, df_f_game, 85), axis=1).values,
    ["match_name", "homeTeamID", "awayTeamID",'home_name'],
].values
# ---------------------------------------------------------------------------------------------------------


F:\WPy64-31040\python-3.10.4.amd64\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [145]:
#----------------------------------------------------------

# un nuevo dataframe con los id de clubes de df_game 
# con los id de clubes de df2_clubs, primero junte los 
# equipos en home con las columnas de id de los dos dataframes



club_agrupados_home = df2_games_top5[
    ["home_club_id", "club_home_pretty_name", "homeTeamID"]
]

gb_club_agrupados_home = club_agrupados_home.groupby(["home_club_id", "homeTeamID"])[
    "club_home_pretty_name"
].size()

gb_club_agrupados_home = (
    gb_club_agrupados_home.reset_index(level="home_club_id")
    .groupby("home_club_id")["club_home_pretty_name"]
    .idxmax()
    .reset_index(name="homeTeamID")
)
#---------------------------------------------------------

df2_clubs_top5_only_names = df2_clubs_top5[["club_id", "pretty_name"]]


gb_club_agrupados_home = pd.merge(
    gb_club_agrupados_home,
    df2_clubs_top5_only_names,
    left_on="home_club_id",
    right_on="club_id",
)


gb_club_agrupados_home = pd.merge(
    gb_club_agrupados_home, df_teams, left_on="homeTeamID", right_on="teamID"
)

# cree un dataset donde estan los nombres de los clubes con su id de los dos datasets df_teams y df2_clubes
gb_club_agrupados_home = gb_club_agrupados_home.drop(
    columns=["home_club_id", "homeTeamID", "pretty_name", "name"]
)



In [146]:
df_teams=pd.merge(df_teams,gb_club_agrupados_home,)
df_teams=pd.merge(df_teams,df2_clubs_top5,on='club_id')


In [147]:
df_teams

,teamID,name_x,club_id,name_y,pretty_name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,url
0,71,Aston Villa,405,aston-villa,Aston Villa,GB1,NaN,26,27.2,17,65.4,14,Villa Park,42682,€-28.00m,Unai Emery,https://www.transfermarkt.co.uk/aston-villa/st...
1,72,Everton,29,fc-everton,Fc Everton,GB1,NaN,27,27.6,13,48.1,11,Goodison Park,39571,€-20.20m,Frank Lampard,https://www.transfermarkt.co.uk/fc-everton/sta...
2,74,Southampton,180,fc-southampton,Fc Southampton,GB1,NaN,24,25.0,16,66.7,11,St Mary's Stadium,32384,€-79.40m,Nathan Jones,https://www.transfermarkt.co.uk/fc-southampton...
3,75,Leicester,1003,leicester-city,Leicester City,GB1,NaN,26,27.3,17,65.4,16,King Power Stadium,32273,+€64.40m,Brendan Rodgers,https://www.transfermarkt.co.uk/leicester-city...
4,76,West Bromwich Albion,984,west-bromwich-albion,West Bromwich Albion,GB1,77.90,26,26.3,9,34.6,6,The Hawthorns,26850,+£11.20m,Slaven Bilic,https://www.transfermarkt.co.uk/west-bromwich-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,182,Nancy,1159,as-nancy-lorraine,As Nancy Lorraine,FR1,16.88,30,24.2,16,53.3,4,Stade Marcel-Picot,20087,+-0,Pablo Correa,https://www.transfermarkt.co.uk/as-nancy-lorra...
142,224,Amiens,1416,amiens-sc,Amiens Sc,FR1,20.97,27,24.7,13,48.1,6,Stade Crédit Agricole la Licorne,12999,+£5.06m,Luka Elsner,https://www.transfermarkt.co.uk/amiens-sc/star...
143,225,Strasbourg,667,rc-strassburg-alsace,Rc Strassburg Alsace,FR1,NaN,25,26.4,9,36.0,6,Stade de la Meinau,29000,€-3.50m,Julien Stéphan,https://www.transfermarkt.co.uk/rc-strassburg-...
144,235,Nimes,1160,nimes-olympique,Nimes Olympique,FR1,27.95,25,24.0,12,48.0,7,Stade des Costières,18364,+£5.22m,Jérôme Arpinon,https://www.transfermarkt.co.uk/nimes-olympiqu...


In [148]:
df_f_game

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,...,fouls_away,corners_away,yellowCards_away,redCards_away,result_away,liga,home_name,away_name,month_year,match_name
0,81.0,1.0,2015.0,2015-08-08,89.0,82.0,1.0,0.0,0.2843,0.3999,...,12.0,2.0,3.0,0.0,L,Premier League,Manchester United,Tottenham,2015-08,Manchester United vs Tottenham
1,82.0,1.0,2015.0,2015-08-08,73.0,71.0,0.0,1.0,0.3574,0.3500,...,13.0,3.0,4.0,0.0,W,Premier League,Bournemouth,Aston Villa,2015-08,Bournemouth vs Aston Villa
2,83.0,1.0,2015.0,2015-08-08,72.0,90.0,2.0,2.0,0.2988,0.4337,...,13.0,2.0,2.0,0.0,D,Premier League,Everton,Watford,2015-08,Everton vs Watford
3,84.0,1.0,2015.0,2015-08-08,75.0,77.0,4.0,2.0,0.6422,0.2057,...,17.0,3.0,4.0,0.0,L,Premier League,Leicester,Sunderland,2015-08,Leicester vs Sunderland
4,85.0,1.0,2015.0,2015-08-08,79.0,78.0,1.0,3.0,0.1461,0.2159,...,20.0,4.0,0.0,0.0,W,Premier League,Norwich,Crystal Palace,2015-08,Norwich vs Crystal Palace
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,16132.0,5.0,2020.0,2021-05-23,177.0,176.0,1.0,2.0,0.3367,0.2999,...,6.0,2.0,1.0,0.0,W,Ligue 1,Reims,Bordeaux,2021-05,Reims vs Bordeaux
12677,16133.0,5.0,2020.0,2021-05-23,163.0,235.0,2.0,0.0,0.6719,0.2502,...,10.0,3.0,0.0,0.0,L,Ligue 1,Rennes,Nimes,2021-05,Rennes vs Nimes
12678,16134.0,5.0,2020.0,2021-05-23,175.0,181.0,0.0,1.0,0.3541,0.3010,...,10.0,3.0,0.0,0.0,W,Ligue 1,Saint-Etienne,Dijon,2021-05,Saint-Etienne vs Dijon
12679,16135.0,5.0,2020.0,2021-05-23,225.0,179.0,1.0,1.0,0.1748,0.4863,...,9.0,3.0,1.0,0.0,D,Ligue 1,Strasbourg,Lorient,2021-05,Strasbourg vs Lorient


In [149]:
df2_games_top5

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,name,confederation,url_y,liga_new_id,month_year,match_name,match_name_df,homeTeamID,awayTeamID,home_name
0,2475559,ES1,domestic_league,2014,38. Matchday,2015-05-23,1084,368,2,3,...,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Fc Malaga vs Fc Sevilla,Malaga vs Sevilla,137.0,138.0,Malaga
1,2475556,ES1,domestic_league,2014,38. Matchday,2015-05-23,418,3709,7,3,...,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Real Madrid vs Fc Getafe,Real Madrid vs Getafe,150.0,142.0,Real Madrid
2,2475555,ES1,domestic_league,2014,38. Matchday,2015-05-23,1533,993,3,0,...,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Sd Eibar vs Fc Cordoba,Eibar vs Cordoba,156.0,206.0,Eibar
3,2475557,ES1,domestic_league,2014,38. Matchday,2015-05-23,940,714,3,2,...,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Celta Vigo vs Espanyol Barcelona,Celta Vigo vs Espanyol,152.0,141.0,Celta Vigo
4,2475558,ES1,domestic_league,2014,38. Matchday,2015-05-23,3302,1049,2,3,...,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Ud Almeria vs Fc Valencia,Almeria vs Valencia,208.0,146.0,Almeria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,3433222,IT1,domestic_league,2020,4. Matchday,2020-10-18,416,1390,2,3,...,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Torino vs Cagliari,Torino vs Cagliari,113.0,116.0,Torino
12677,3433216,IT1,domestic_league,2020,4. Matchday,2020-10-17,4083,506,1,1,...,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Fc Crotone vs Juventus,Crotone vs Juventus,115.0,98.0,Crotone
12678,3433190,IT1,domestic_league,2020,1. Matchday,2020-09-21,5,1025,2,0,...,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-09,Ac Milan vs Fc Bologna,AC Milan vs Bologna,111.0,97.0,AC Milan
12679,3433206,IT1,domestic_league,2020,3. Matchday,2020-10-04,4171,1025,1,0,...,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Benevento Calcio vs Fc Bologna,Atalanta vs Cagliari,107.0,116.0,Atalanta


In [150]:

#cree un dataset separado de df_games y df2_games_top5 donde solo tengo las columnas de id y algunos datos

df2_games_top5_game_id=df2_games_top5[['game_id','date','month_year','home_club_id','away_club_id']]
df_f_game_game_id=df_f_game[['gameID','date','month_year','homeTeamID','awayTeamID']]
df_f_game_game_id.drop(index=12680,inplace=True)
#---------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------
#df2_games_top5 lo uno con el datasets de clubes donde ahora tiene los id del teams del datasets df_teams para poder hacer el merge asof
df2_games_top5_game_id=pd.merge(df2_games_top5_game_id,gb_club_agrupados_home,left_on='home_club_id',right_on='club_id')
df2_games_top5_game_id=pd.merge(df2_games_top5_game_id,gb_club_agrupados_home,left_on='away_club_id',right_on='club_id',suffixes=('_home','_away'))
df2_games_top5_game_id.drop(columns=['club_id_home','club_id_away'],inplace=True)
df2_games_top5_game_id.rename(columns={'teamID_home':"homeTeamID",'teamID_away':'awayTeamID'},inplace=True)


#--------------------------------------------
#preparo el datasets df_f_game
df_f_game_game_id['date']=pd.to_datetime(df_f_game_game_id.date)
df_f_game_game_id=df_f_game_game_id.sort_values(by='date')
df2_games_top5_game_id=df2_games_top5_game_id.sort_values(by='date')
df_f_game_game_id['homeTeamID']=df_f_game_game_id.homeTeamID.astype('int64')
df_f_game_game_id['awayTeamID']=df_f_game_game_id.awayTeamID.astype('int64')


#--------------------------------------------
#ahora uno el con la fecha mas cercana los df2_games y df_f_game y segun las columnas de hometeam y away
df2_games_top5_game_id=pd.merge_asof(df2_games_top5_game_id,df_f_game_game_id,on='date', by=['homeTeamID','awayTeamID'], direction='nearest')


df_f_game_game_id_date=df2_games_top5_game_id[['gameID','game_id','date']]
df_appearances=pd.merge(df_appearances,df_f_game_game_id_date)

df_appearances=pd.merge(df_appearances,df_players)





In [151]:
df2_appearances


,appearance_id,game_id,player_id,player_club_id,date,player_pretty_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
34,2486513_37787,2486513,37787,2288,2014-09-20,Marvin Emnes,GB1,0,0,0,0,5
35,2460649_119557,2460649,119557,42,2014-09-14,Kenan Karaman,L1,0,0,0,0,19
64,2460300_22388,2460300,22388,2288,2014-08-16,Bafetimbi Gomis,GB1,0,0,0,0,13
75,2486677_72768,2486677,72768,2288,2015-01-10,Nelson Oliveira,GB1,0,0,0,0,16
76,2486686_72768,2486686,72768,2288,2015-01-17,Nelson Oliveira,GB1,0,0,0,0,66
...,...,...,...,...,...,...,...,...,...,...,...,...
905941,3433343_167105,3433343,167105,252,2021-01-06,Lukas Lerager,IT1,0,0,0,0,90
905942,3433347_167105,3433347,167105,252,2021-01-09,Lukas Lerager,IT1,0,0,0,0,16
905943,3433355_167105,3433355,167105,252,2021-01-17,Lukas Lerager,IT1,0,0,0,0,45
907464,3394631_709726,3394631,709726,1421,2020-10-17,Hugo Ekitike,FR1,1,0,0,0,55


In [152]:
m = df2_appearances.select_dtypes(np.number)
df2_appearances[m.columns]= m.round().astype('Int64')

m = df_appearances.select_dtypes(np.number)
df_appearances[m.columns]= m.round().astype('Int64')


In [153]:
df2_appearances.drop(columns='player_pretty_name',inplace=True)
df2_col=df2_players[['player_id','pretty_name']]


In [154]:
df2_appearances=pd.merge(df2_appearances,df2_col)

#-----------------------------------------------
#cree una fila de nan para usarla en el match
nan_values = pd.DataFrame(np.nan, index=[0], columns=df2_appearances.columns)
df2_appearances = df2_appearances.append(nan_values, ignore_index=True)

C:\Users\Mati\AppData\Local\Temp\ipykernel_2024\2386021328.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2_appearances = df2_appearances.append(nan_values, ignore_index=True)


In [155]:
df2_appearances.tail()

,appearance_id,game_id,player_id,player_club_id,date,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,pretty_name
355323,3450983_199299,3450983.0,199299.0,1533.0,2020-10-30,ES1,0.0,0.0,0.0,0.0,15.0,Damian Kadzior
355324,3451062_199299,3451062.0,199299.0,1533.0,2020-12-29,ES1,0.0,0.0,0.0,0.0,81.0,Damian Kadzior
355325,3394631_709726,3394631.0,709726.0,1421.0,2020-10-17,FR1,1.0,0.0,0.0,0.0,55.0,Hugo Ekitike
355326,3394635_709726,3394635.0,709726.0,1421.0,2020-10-25,FR1,0.0,0.0,0.0,0.0,31.0,Hugo Ekitike
355327,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,NaN


In [156]:
df_appearances.tail()

,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,...,positionOrder,yellowCard,redCard,time,substituteIn,substituteOut,leagueID,game_id,date,name
356500,16132,7396,0,0,0,0,0,0,0,0,...,3,0,0,90,0,0,5,3394999,2021-05-23,Loic Bessile
356501,16134,9566,0,0,1,0,0,0,0,0,...,17,0,0,29,0,474027,5,3395001,2021-05-23,Yanis Lhï¿½ry
356502,16134,9565,0,0,0,0,0,0,0,0,...,17,0,0,12,0,474032,5,3395001,2021-05-23,Mathys Saban
356503,16134,9568,0,0,0,0,0,0,0,0,...,17,0,0,1,0,474045,5,3395001,2021-05-23,Charles Costes
356504,16134,9567,0,0,1,0,0,0,0,1,...,17,0,0,29,0,474043,5,3395001,2021-05-23,Erwan Belhadji


In [157]:
"""   yellowCard = x["yellowCard"]
    redCard = x["redCard"]
    goals = x["goals"]
    assists = x["assists"] 
 & (y["yellow_cards"] == yellowCard)
        & (y["red_cards"] == redCard)
        & (y["goals"] == goals)
        & (y["assists"] == assists """

def match_games(x, y, score):

    game = x["game_id"]


    y = y.loc[y["game_id"] == game]

    match = process.extractOne(
        x["name"], y["pretty_name"], score_cutoff=score, scorer=fuzz.WRatio
    )

    if match:
        return match[2]
    else:
        return 355327


# ---------------------------------------------------------------------------------------------------------

df_appearances[['playerID_df2', "player_club_id", "appearance_id",'pretty_name']] = df2_appearances.loc[
    df_appearances.apply(lambda x: match_games(x, df2_appearances, 85), axis=1).values,
    ['player_id', "player_club_id", "appearance_id",'pretty_name'],
].values
# ---------------------------------------------------------------------------------------------------------


In [167]:
df_appearances = df_appearances[df_appearances['playerID_df2'].notna()]

In [168]:
df_appearances.isna().sum()

gameID            0
playerID          0
goals             0
ownGoals          0
shots             0
xGoals            0
xGoalsChain       0
xGoalsBuildup     0
assists           0
keyPasses         0
xAssists          0
position          0
positionOrder     0
yellowCard        0
redCard           0
time              0
substituteIn      0
substituteOut     0
leagueID          0
game_id           0
date              0
name              0
playerID_df2      0
player_club_id    0
appearance_id     0
pretty_name       0
dtype: int64

In [169]:
club_agrupados_players = df_appearances[
    ["playerID", "player_club_id", "playerID_df2"]
]

gb_club_agrupados_players = club_agrupados_players.groupby(["playerID", "playerID_df2"])[
    "player_club_id"
].size()

gb_club_agrupados_players = (
    gb_club_agrupados_players.reset_index(level="playerID")
    .groupby("playerID")["player_club_id"]
    .idxmax()
    .reset_index(name="playerID_df2")
)
df2_players=df2_players[
    df2_players["player_id"].isin(df2_appearances["player_id"])
]


df_players=pd.merge(df_players,gb_club_agrupados_players,on='playerID',how='left')
gb_club_agrupados_players=pd.merge(gb_club_agrupados_players,df_players)

gb_club_agrupados_players=pd.merge(gb_club_agrupados_players,df2_players,left_on='playerID_df2',right_on='player_id')


In [170]:


df_appearances.isna().sum()

gameID            0
playerID          0
goals             0
ownGoals          0
shots             0
xGoals            0
xGoalsChain       0
xGoalsBuildup     0
assists           0
keyPasses         0
xAssists          0
position          0
positionOrder     0
yellowCard        0
redCard           0
time              0
substituteIn      0
substituteOut     0
leagueID          0
game_id           0
date              0
name              0
playerID_df2      0
player_club_id    0
appearance_id     0
pretty_name       0
dtype: int64

In [171]:

df_teams['total_market_value']=df_teams['total_market_value'].fillna(df_teams['total_market_value'].median())
df_teams.isna().sum()

teamID                     0
name_x                     0
club_id                    0
name_y                     0
pretty_name                0
domestic_competition_id    0
total_market_value         0
squad_size                 0
average_age                1
foreigners_number          0
foreigners_percentage      1
national_team_players      0
stadium_name               0
stadium_seats              0
net_transfer_record        0
coach_name                 0
url                        0
dtype: int64

In [172]:


gb_club_agrupados_players.rename(columns={'name_x':'name'},inplace=True)

gb_club_agrupados_players.drop(columns=['name_y'],inplace=True)
gb_club_agrupados_players['name']=gb_club_agrupados_players["name"].str.replace(";","")

In [173]:
df_f_game.to_csv('df_game_new.csv',index=False)
df_appearances.to_csv('df_appearances_new.csv',index=False)
df_teams.to_csv('df_teams_new.csv',index=False)
gb_club_agrupados_players.to_csv('df_players_new.csv',index=False)

In [174]:
gb_club_agrupados_players.loc[gb_club_agrupados_players["name"]=="Messi"]

,playerID,playerID_df2,name,playerID_df2_x,playerID_df2_y,player_id,pretty_name,club_id,club_pretty_name,current_club_id,...,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,agent_name,contract_expiration_date,domestic_competition_id,club_name,image_url,last_season,url
1803,2097,28003.0,Messi,28003.0,28003.0,28003,Messi,583,Fc Paris Saint Germain,583,...,170,50000000.0,180000000.0,NaN,2023-06-30,FR1,fc-paris-saint-germain,https://img.a.transfermarkt.technology/portrai...,2022,https://www.transfermarkt.co.uk/lionel-messi/p...
